In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
print('DONE')


DONE


In [2]:
pip install ultralytics

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os
import torch
from tqdm import tqdm
from torchvision import models, transforms
from ultralytics import YOLO
import random

# Load YOLO model
yolo_model_path = "/kaggle/input/yolov5npt/yolov5su.pt"
yolo_model = YOLO(yolo_model_path)

# Load a pre-trained CNN (ResNet50) for feature extraction
cnn_model = models.resnet50(pretrained=True)
cnn_model.eval()

# Image preprocessing pipeline for CNN input
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Output paths for extracted frames
output_paths = {
    "train_violence": "/kaggle/working/extracted_frames/train/violence",
    "train_nonviolence": "/kaggle/working/extracted_frames/train/nonviolence",
    "val_violence": "/kaggle/working/extracted_frames/val/violence",
    "val_nonviolence": "/kaggle/working/extracted_frames/val/nonviolence"
}

# Create output directories if they don't exist
for path in output_paths.values():
    os.makedirs(path, exist_ok=True)

# Function to extract YOLO-detected bounding box regions
def extract_yolo_frames(video_path, output_folder, target_fps=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps == 0:
        print(f"Error reading FPS for video: {video_path}")
        return

    frame_interval = int(original_fps / target_fps)
    frame_count, saved_frame_count = 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            # YOLO detection
            results = yolo_model(frame)
            for bbox in results[0].boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = bbox
                if score > 0.5:  # Confidence threshold
                    cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
                    if cropped_frame.size > 0:
                        # Save cropped region
                        crop_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_crop{saved_frame_count}.jpg"
                        cv2.imwrite(os.path.join(output_folder, crop_filename), cropped_frame)
                        saved_frame_count += 1
        frame_count += 1
    cap.release()

# Function to extract features from cropped images
def extract_features_from_images(image_folder):
    features = {}
    for img_file in tqdm(os.listdir(image_folder), desc="Extracting CNN features"):
        img_path = os.path.join(image_folder, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            feature_vector = cnn_model(img_tensor)
        features[img_file] = feature_vector.squeeze().numpy()  # Store feature vector
    return features

# Process videos to extract YOLO bounding boxes
def process_videos(video_paths, train_output, val_output, split_for_val=False, target_fps=1):
    for folder in video_paths:
        video_files = [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        if split_for_val:
            random.shuffle(video_files)
            val_count = max(1, int(len(video_files) * 0.1))  # Take 10% for validation
            val_videos, train_videos = video_files[:val_count], video_files[val_count:]
        else:
            val_videos, train_videos = [], video_files

        # Extract YOLO bounding boxes for validation videos
        for video_file in tqdm(val_videos, desc=f"Processing validation videos from {folder}"):
            extract_yolo_frames(os.path.join(folder, video_file), val_output, target_fps=target_fps)

        # Extract YOLO bounding boxes for training videos
        for video_file in tqdm(train_videos, desc=f"Processing training videos from {folder}"):
            extract_yolo_frames(os.path.join(folder, video_file), train_output, target_fps=target_fps)

# Process videos sequentially
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)

print("YOLO-based bounding box extraction completed.")

# Extract features from cropped images sequentially
train_violence_features = extract_features_from_images(output_paths["train_violence"])
train_nonviolence_features = extract_features_from_images(output_paths["train_nonviolence"])
val_violence_features = extract_features_from_images(output_paths["val_violence"])
val_nonviolence_features = extract_features_from_images(output_paths["val_nonviolence"])

print("Feature extraction completed.")


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing validation videos from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence:   0%|          | 0/100 [00:00<?, ?it/s]


0: 640x640 1 person, 2 cars, 9.0ms
Speed: 9.6ms preprocess, 9.0ms inference, 153.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 7.5ms
Speed: 2.8ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 persons, 2 cars, 7.5ms
Speed: 3.9ms preprocess, 7.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 persons, 7.4ms
Speed: 2.5ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


Processing validation videos from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence:   1%|          | 1/100 [00:00<01:33,  1.05it/s]


0: 640x640 1 person, 7.8ms
Speed: 2.8ms preprocess, 7.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 car, 7.5ms
Speed: 2.0ms preprocess, 7.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 7.5ms
Speed: 2.6ms preprocess, 7.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 2.7ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 2 bicycles, 7.5ms
Speed: 1.9ms preprocess, 7.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing validation videos from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence:   2%|▏         | 2/100 [00:01<00:44,  2.22it/s]

...


0: 384x640 5 persons, 9.4ms
Speed: 3.1ms preprocess, 9.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 chair, 10.3ms
Speed: 3.2ms preprocess, 10.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 1 suitcase, 8.2ms
Speed: 2.9ms preprocess, 8.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 1 train, 9.1ms
Speed: 3.2ms preprocess, 9.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 8.7ms
Speed: 3.1ms preprocess, 8.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing validation videos from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence:  25%|██▌       | 25/100 [00:05<00:27,  2.75it/s]


0: 640x640 4 persons, 8.2ms
Speed: 3.1ms preprocess, 8.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 bottle, 8.2ms
Speed: 2.0ms preprocess, 8.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.4ms
Speed: 2.7ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 7.5ms
Speed: 2.7ms preprocess, 7.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 parking meter, 7.4ms
Speed: 1.9ms preprocess, 7.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x416 1 person, 2 skateboards, 1 toilet, 1 sink, 40.7ms
Speed: 2.7ms preprocess, 40.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 416)

0: 640x416 2 persons, 1 cell phone, 6.4ms
Speed: 2.4ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 416)


Processing validation videos from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence:  26%|██▌       | 26/100 [00:06<00:17,  4.23it/s]


KeyboardInterrupt: 

In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the directories containing extracted frames for training and validation
train_dir = "/kaggle/working/extracted_frames/train"
val_dir = "/kaggle/working/extracted_frames/val"

# Parameters
IMG_SIZE = (224, 224)  # Target image size for resizing
BATCH_SIZE = 32  # Number of images per batch during training/validation

# 1. Data Augmentation and Preprocessing
# Training Data Augmentation (train_datagen)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values from [0, 255] to [0, 1]
    rotation_range=20,  # Random rotation within ±20 degrees
    width_shift_range=0.2,  # Random horizontal shift (20% of image width)
    height_shift_range=0.2,  # Random vertical shift (20% of image height)
    shear_range=0.2,  # Random shearing transformations
    zoom_range=0.2,  # Random zoom (in or out) by up to 20%
    horizontal_flip=True,  # Random horizontal flip
    brightness_range=[0.8, 1.2],  # Randomly adjust image brightness
    fill_mode='nearest'  # Handle pixels outside image boundary using nearest pixel value
)

# Validation Data Preprocessing (val_datagen)
val_datagen = ImageDataGenerator(
    rescale=1.0/255  # Only rescale pixel values for validation (no augmentation)
)

# 2. Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,  # Resize images to the target size
    batch_size=BATCH_SIZE,  # Group images into batches of size BATCH_SIZE
    class_mode='binary',  # Binary classification (two classes: 0 or 1)
    shuffle=True  # Shuffle the data for training to ensure randomness
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,  # Resize images to the target size
    batch_size=BATCH_SIZE,  # Group images into batches of size BATCH_SIZE
    class_mode='binary',  # Binary classification for validation as well
    shuffle=False  # Do not shuffle for validation, keeping consistent ordering
)

# These generators can now be used in a model's fit method for training and validation.
# Example: cnn_model.fit(train_generator, validation_data=val_generator, epochs=10)


Found 74316 images belonging to 2 classes.
Found 36154 images belonging to 2 classes.


In [5]:
import os

# Specify the path to the main directory
main_dir = '/kaggle/working/extracted_frames'

# Dictionary to store file counts
file_counts = {}

# Walk through each subdirectory and count files
for root, dirs, files in os.walk(main_dir):
    # Only count files, skip directories
    file_counts[root] = len(files)

# Print the file counts in each folder
for folder, count in file_counts.items():
    print(f"{folder}: {count} files")


/kaggle/working/extracted_frames: 0 files
/kaggle/working/extracted_frames/train: 0 files
/kaggle/working/extracted_frames/train/nonviolence: 32940 files
/kaggle/working/extracted_frames/train/violence: 41376 files
/kaggle/working/extracted_frames/val: 0 files
/kaggle/working/extracted_frames/val/nonviolence: 14523 files
/kaggle/working/extracted_frames/val/violence: 21631 files


In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm import tqdm

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Data loaders
train_dataset = datasets.ImageFolder('/kaggle/working/extracted_frames/train', transform=transform)
val_dataset = datasets.ImageFolder('/kaggle/working/extracted_frames/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# Training loop
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels.unsqueeze(1)).sum().item()
        total += labels.size(0)
    
    return running_loss/len(loader), correct/total

# Start training
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    loss, acc = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}: Loss = {loss:.4f}, Acc = {acc:.4f}')

100%|██████████| 2323/2323 [02:08<00:00, 18.14it/s]


Epoch 1: Loss = 0.4085, Acc = 0.8162


100%|██████████| 2323/2323 [02:09<00:00, 18.00it/s]


Epoch 2: Loss = 0.3947, Acc = 0.8263


100%|██████████| 2323/2323 [02:16<00:00, 17.04it/s]


...

Epoch 8: Loss = 0.3763, Acc = 0.8368


100%|██████████| 2323/2323 [02:10<00:00, 17.73it/s]


Epoch 9: Loss = 0.3703, Acc = 0.8408


100%|██████████| 2323/2323 [02:12<00:00, 17.59it/s]

Epoch 10: Loss = 0.3700, Acc = 0.8413


In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Total trainable parameters: {count_parameters(model):,}')
print("\nModel Architecture:")
print(model)

Total trainable parameters: 3,659,017

Model Architecture:
MobileNetClassifier(
  (base_model): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (activation): ReLU()
       

In [19]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        
        self.optimizer = Adam(model.parameters(), lr=1e-4)
        self.criterion = torch.nn.BCELoss()
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-5)
        
    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in tqdm(self.train_loader):
            inputs, labels = inputs.to(self.device), labels.to(self.device).float()
            
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels.unsqueeze(1))
            
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels.unsqueeze(1)).sum().item()
            total += labels.size(0)
            
        return running_loss/len(self.train_loader), correct/total
    
    def validate(self):
        self.model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(self.val_loader):
                inputs, labels = inputs.to(self.device), labels.to(self.device).float()
                
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels.unsqueeze(1))
                
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                correct += (predicted == labels.unsqueeze(1)).sum().item()
                total += labels.size(0)
                
        return val_loss/len(self.val_loader), correct/total

    def train(self, epochs=15):
        best_val_loss = float('inf')
        
        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()
            
            # Learning rate scheduling
            self.scheduler.step(val_loss)
            
            # Model checkpoint
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'loss': val_loss,
                }, 'best_model.pth')
                print(f'Saved checkpoint at epoch {epoch+1}')
            
            print(f'Epoch {epoch+1}/{epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
            print(f'Current LR: {self.optimizer.param_groups[0]["lr"]:.6f}\n')

# Usage
trainer = ModelTrainer(model, train_loader, val_loader, device)
trainer.train(epochs=5)

100%|██████████| 1130/1130 [01:06<00:00, 16.92it/s]


Saved checkpoint at epoch 1
Epoch 1/5
Train Loss: 0.3034, Train Acc: 0.8732
Val Loss: 0.4141, Val Acc: 0.8426
Current LR: 0.000100



100%|██████████| 1130/1130 [01:11<00:00, 15.83it/s]


Epoch 2/5
Train Loss: 0.2908, Train Acc: 0.8793
Val Loss: 0.4514, Val Acc: 0.8381
Current LR: 0.000100



100%|██████████| 1130/1130 [01:10<00:00, 16.03it/s]


Epoch 3/5
Train Loss: 0.2820, Train Acc: 0.8829
Val Loss: 0.4652, Val Acc: 0.8409
Current LR: 0.000100



100%|██████████| 1130/1130 [01:06<00:00, 17.03it/s]


Epoch 4/5
Train Loss: 0.2772, Train Acc: 0.8858
Val Loss: 0.4461, Val Acc: 0.8485
Current LR: 0.000100



100%|██████████| 1130/1130 [01:09<00:00, 16.27it/s]

Epoch 5/5
Train Loss: 0.2737, Train Acc: 0.8874
Val Loss: 0.5169, Val Acc: 0.8505
Current LR: 0.000100



In [28]:
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torch.optim import Adam
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os

# Define the MobileNetClassifier
class MobileNetClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # Load the base model with pre-trained weights
        self.base_model = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
        
        # Freeze the base model if needed
        for param in self.base_model.parameters():
            param.requires_grad = True
        
        # Modify final layers
        self.features = nn.Sequential(
            self.base_model.features,
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(576, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# Create model and move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MobileNetClassifier().to(device)

# Setup optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.01)  # L2 regularization
criterion = nn.BCELoss()

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # MobileNetV3 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset (adjusted to your directory structure)
data_dir = "/kaggle/working/extracted_frames"
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Training loop with model saving
num_epochs = 5
best_val_loss = float('inf')  # Initialize to a very large value

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
            
            # Calculate accuracy
            predicted = (outputs.squeeze() > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")
    
    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "/kaggle/working/best_model.pth")
        print("Saved best model.")

# Load the best model for further use or testing
model.load_state_dict(torch.load("/kaggle/working/best_model.pth"))
model.to(device)
print("Loaded the best model.")


Epoch [1/5], Loss: 0.4137
Validation Loss: 0.7788, Accuracy: 52.82%
Saved best model.
Epoch [2/5], Loss: 0.3923
Validation Loss: 0.8648, Accuracy: 63.38%
Epoch [3/5], Loss: 0.3858
Validation Loss: 0.4851, Accuracy: 77.69%
Saved best model.
Epoch [4/5], Loss: 0.3838
Validation Loss: 0.4776, Accuracy: 78.90%
Saved best model.
Epoch [5/5], Loss: 0.3797
Validation Loss: 0.5114, Accuracy: 75.84%
Loaded the best model.


In [29]:
from torchinfo import torchinfo

# Print model summary with sample input size
summary = torchinfo.summary(model, 
                          input_size=(1, 3, 224, 224),
                          col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
                          depth=4)
print(summary)

# If you don't have torchinfo installed, run:
# !pip install torchinfo

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
MobileNetClassifier                                     [1, 3, 224, 224]          [1, 1]                    1,615,848                 --                        --
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 576]                  --                        --                        --
│    └─Sequential: 2-1                                  [1, 3, 224, 224]          [1, 576, 7, 7]            --                        --                        --
│    │    └─Conv2dNormActivation: 3-1                   [1, 3, 224, 224]          [1, 16, 112, 112]         --                        --                        --
│    │    │    └─Conv2d: 4-1                            [1, 3, 224, 224]          [1, 16, 112, 112]         432                       [3, 3]                    5,419,008
│    │  

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Load the base model
base_model = MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,  # Exclude the original classifier head
    weights='imagenet'  # Use ImageNet weights
)

# Freeze the base model
base_model.trainable = True  # Set to False if you don't want to fine-tune

# Add custom layers for binary classification
model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Fine-tuning with a lower learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Data preprocessing using ImageDataGenerator
data_dir = "/kaggle/working/extracted_frames"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    directory=f"{data_dir}/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    directory=f"{data_dir}/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Define callbacks
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[checkpoint, reduce_lr]
)

# Load the best model
model = tf.keras.models.load_model('best_model.keras')
print("Best model loaded successfully.")


Found 74316 images belonging to 2 classes.
Found 36154 images belonging to 2 classes.
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1735755916.074809   47966 service.cc:145] XLA service 0x7ac728003f00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735755916.074870   47966 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   1/2323 ━━━━━━━━━━━━━━━━━━━━ 26:28:00 41s/step - accuracy: 0.5000 - loss: 0.7422

I0000 00:00:1735755936.047936   47966 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2322/2323 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.7578 - loss: 0.4847
Epoch 1: val_loss improved from inf to 0.72619, saving model to best_model.keras
2323/2323 ━━━━━━━━━━━━━━━━━━━━ 856s 351ms/step - accuracy: 0.7579 - loss: 0.4846 - val_accuracy: 0.5142 - val_loss: 0.7262 - learning_rate: 1.0000e-04
Epoch 2/5
2322/2323 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step - accuracy: 0.8930 - loss: 0.2496
Epoch 2: val_loss did not improve from 0.72619
2323/2323 ━━━━━━━━━━━━━━━━━━━━ 769s 330ms/step - accuracy: 0.8930 - loss: 0.2496 - val_accuracy: 0.4111 - val_loss: 1.9777 - learning_rate: 1.0000e-04
Epoch 3/5
2322/2323 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.9198 - loss: 0.1940
Epoch 3: val_loss did not improve from 0.72619
2323/2323 ━━━━━━━━━━━━━━━━━━━━ 775s 333ms/step - accuracy: 0.9198 - loss: 0.1940 - val_accuracy: 0.5938 - val_loss: 1.7396 - learning_rate: 1.0000e-04
Epoch 4/5
2322/2323 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.9373 - loss: 0.1577
Epoch 4: val_loss did not 

In [6]:
# Save the model in .keras format
model.save('trained_model.keras')

print("Model saved successfully in .keras format.")


Model saved successfully in .keras format.


In [4]:
pip install tf2onnx


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 9.7 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import tensorflow as tf
import tf2onnx
import os
import numpy as np

def convert_keras_to_onnx(keras_model_path, onnx_model_path):
    """
    Convert a Keras model to ONNX format
    
    Args:
        keras_model_path (str): Path to the Keras model file (.keras)
        onnx_model_path (str): Desired output path for the ONNX model
        
    Returns:
        bool: True if conversion was successful, False otherwise
    """
    try:
        # Load the trained Keras model
        model = tf.keras.models.load_model(keras_model_path)
        
        # Get input shape from the model's configuration
        input_shape = model.layers[0].input_shape
        if input_shape[0] is None:
            # Replace None batch dimension with 1
            input_shape = (1,) + input_shape[1:]
        
        # Create a dummy input to initialize the model
        dummy_input = np.zeros(input_shape, dtype=np.float32)
        
        # Call the model once to initialize it
        _ = model(dummy_input)
        
        # Save as SavedModel format first
        temp_saved_model_path = "temp_saved_model"
        tf.saved_model.save(model, temp_saved_model_path)
        
        # Convert to ONNX
        output_path = onnx_model_path
        command = f"python -m tf2onnx.convert --saved-model {temp_saved_model_path} --output {output_path}"
        os.system(command)
        
        # Clean up temporary directory
        if os.path.exists(temp_saved_model_path):
            import shutil
            shutil.rmtree(temp_saved_model_path)
            
        print(f"Model successfully converted and saved to: {onnx_model_path}")
        return True
        
    except Exception as e:
        print(f"Error during conversion: {str(e)}")
        # Print more detailed error information
        import traceback
        print(traceback.format_exc())
        return False

# Example usage
if __name__ == "__main__":
    keras_model_path = 'trained_model.keras'
    onnx_model_path = 'trained_model.onnx'
    
    success = convert_keras_to_onnx(keras_model_path, onnx_model_path)
    
    if success:
        # Verify the ONNX model exists
        if os.path.exists(onnx_model_path):
            print(f"ONNX model size: {os.path.getsize(onnx_model_path) / (1024*1024):.2f} MB")
        else:
            print("Warning: ONNX file was not created successfully")

/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2025-01-01 19:57:00,330 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2025-01-01 19:57:02,449 - INFO - Signatures found in model: [serving_default].
2025-01-01 19:57:02,449 - WARNING - '--signature_def' not specified, using first signature: serving_default
2025-01-01 19:57:02,450 - INFO - Output names: ['output_0']
2025-01-01 19:57:03,781 - INFO - Using tensorflow=2.16.1, onnx=1.17.0, tf2onnx=1.16.1/15c810
2025-01-01 19:57:03,781 - INFO - Using opset <onnx, 15>
2025-01-01 19:57:03,876 - INFO - Computed 0 values for constant folding
2025-01-01 19:57:04,123 - INFO - Optimizing ONNX model
2025-01-01 19:57:06,055 - INFO - After optimization: Add -41 (79->38), Const -108 (249->141), GlobalAveragePool +10 (0->10), Identity -2 (

Model successfully converted and saved to: trained_model.onnx
ONNX model size: 4.74 MB
